<h1>Preparing the application image</h1>
______________________________

## Introduction
______________________________

In this tutorial, we're going to discuss how to make an application that is ready to for deployment with BlindBox. We'll explore the technical constraints you'll need to bear in mind for your application and show you our example "Hello World!" app code! 👋

## Pre-requisites
____________________

To follow along with this tutorial and deploy your own BlindBox, you will need to:

+ Have Docker installed in your environment. Here's the [Docker installation guide](https://docs.docker.com/desktop/install/linux-install/).

### Technical constraints

It is important to keep in mind the external calls your application might make, when considering application deployment with BlindBox. This means that, ideally, your application should be **self-sufficient**, so that user's data will be sent to our secure execution environment without being exposed to any additional risks. 

We are currently implementing a whitelisting feature so that data owners can whitelist any legitimate external domains needed for the application. When we launch it, any calls made from the BlindBox to domains not whitelisted will be blocked! This feature is coming soon, but it will still be preferable to **rely on as few external domains as possible**, because the data owner will have to verify all these domains.

## The application code
_______________________

Now, let's take a look at our application code that will be launched on our confidential VM by BlindBox.

```python
import uvicorn
from fastapi import FastAPI

 # initialize our API app object
app = FastAPI()

 # use fastapi decorator to turn our Hello World function into API endpoints on our uvicorn server
@app.get("/hello")
def hello() -> str:
    return "Hello World"

if __name__ == "__main__":
    # deploy our server on port 80
    uvicorn.run(app, host="0.0.0.0", port=80)
```

You can view this file by downloading our github repository and taking a look at the `server.py` file in the `examples/hello-world` folder.

In [ ]:
!git clone https://github.com/mithril-security/blindbox
!cd examples/hello-world

This program does the following:

+ Creates the API app object that "configures" the uvicorn server by providing handlers for specific endpoints

+ Creates a `hello` endpoint on our `uvicorn server` which users will be able to query on our BlindBox address once packaged and deployed.

+ Deploys our API on a python ASGI server (an asynchronous web server) on `port 80`. 

	> It is important that the application is configured to run on **port 80** because BlindBox will forward and receive traffic to the application on this port!

## Packaging our application in a Docker image
________________________________

Now that we're set up, we need to **package** our application in a **Docker image**. We'll take a look at the Dockerfile we used to package this Hello World application:

```docker
FROM python:3.10-alpine

 # download any requisites
RUN pip install 
        # provides handlers for specific endpoints on our server
        fastapi==0.95.0
        # gives support for multipart requests
        python-multipart==0.0.6
        # uvicorn is a python ASGI server (an asynchronous web server)
        uvicorn==0.21.1
        # fastapi uses this library to define response/requests formats and to serialize/deserialize the response/requests
        pydantic==1.10.7

 # copy our application code across to our container
COPY server.py /

 # signal that application will run on port 80
EXPOSE 80

 # set command on launch to launch our application code
CMD ["python", "/server.py"]
```

> Same as for the application code, this file can be viewed in the `examples` folder in the official BlindBox GitHub repository.

There are no complex requirements for the Docker image, but it is recommended to `EXPOSE` port 80 to signal that the application will be running on port 80 within our BlindBox.

## Building our application
_____________________________

But let's build our Hello World application image before we can deploy it with BlindBox:

In [ ]:
# build application assigning it the tag "hello-world"
!docker build -t hello-world .

## Conclusion
_______________________
 
In this tutorial, we've seen how we can:
+ Create a BlindBox-compatible application
+ Package our application

Now that the Docker image is packaged it is ready to be deployed with BlindBox. To see how to deploy an application image with BlindBox, check out our [Quick Tour](https://blindbox.mithrilsecurity.io/en/latest/docs/getting-started/quick-tour/).